## Inicializar la Dimensión Novedad

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

# Añadir conexión a la base de datos


In [6]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Extracción de Datos para la Dimensión Novedad

In [7]:
query = """
SELECT
    mn.id AS novedad_id,
    mn.descripcion AS novedad_descripcion,
    mn.fecha_novedad,
    TO_CHAR(mn.fecha_novedad, 'YYYY-MM-DD') AS novedad_fecha,
    TO_CHAR(mn.fecha_novedad, 'HH24:MI:SS') AS novedad_hora,
    mt.nombre AS tipo_novedad_nombre
FROM
    public.mensajeria_novedadesservicio mn
LEFT JOIN
    public.mensajeria_tiponovedad mt ON mn.tipo_novedad_id = mt.id;
"""

# Leer los datos desde la base de datos 'RAPIDOS-Y_FURIOSOS' a un DataFrame de Pandas
df_novedad = pd.read_sql(query, ryf_conn)

# Mostrar los primeros registros para verificar la extracción
df_novedad.head()

,novedad_id,novedad_descripcion,fecha_novedad,novedad_fecha,novedad_hora,tipo_novedad_nombre
0,4,A,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
1,5,Halo,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
2,6,A,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
3,7,B,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
4,8,A,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio


Transformación de los Datos para la Dimensión Novedad

In [8]:
# Verificar duplicados y datos nulos
df_novedad.drop_duplicates(subset=['novedad_id'], inplace=True)
df_novedad.fillna({'novedad_descripcion': 'Sin descripción'}, inplace=True)

# Mostrar los primeros registros después de la transformación
df_novedad.head()

,novedad_id,novedad_descripcion,fecha_novedad,novedad_fecha,novedad_hora,tipo_novedad_nombre
0,4,A,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
1,5,Halo,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
2,6,A,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
3,7,B,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio
4,8,A,2023-11-30 05:00:00+00:00,2023-11-30,05:00:00,Novedades del servicio


# Carga de la Dimensión Novedad 

In [9]:
df_novedad.to_sql('dim_novedad', etl_conn, if_exists='replace', index_label='key_dim_novedad')


208